# Create Database from Heatpump Keymark
## 1) download from https://keymark.eu/en/products/heatpumps/certified-products
## 2) use dash-skript to convert pdf into txt
## 3) use this skript:




In [2]:
import hplib

In [ ]:
# Import keymark data and save to csv database
hplib.ImportKeymarkData()

In [ ]:
# Reduce to climate measurement series and delete redunent entries and save to csv sub-database
hplib.ReduceKeymarkData('database_keymark.csv','average')

In [3]:
# Normalize electrical and thermal power from the keymark database to values from setpoint T_in = -7 °C and T_out = 52 °C
hplib.NormalizeKeymarkData('database_keymark_average.csv')

In [ ]:
def get_modus(P_thermal1, P_thermal2, P_thermal3, P_thermal4):
    if (P_thermal1 <= P_thermal2):
        if (P_thermal2 <= P_thermal3):
            if (P_thermal3 <= P_thermal4):
                modus='On-Off'#\\\
            else:
                modus='On-Off'#/\\ idk why but its true examples: AQUATOP S08; TTF 35
        else:
            if(P_thermal3 <= P_thermal4):
                modus='2-Stages'#\/\
            else:
                modus='Inverter'#\//
    else:
        #if (P_thermal2 <= P_thermal3):
        #if(P_thermal3 <= P_thermal4):
        #modus='2-stagesLW300'
        modus='Inverter'#/\/
    if (P_thermal1 == P_thermal4):
        modus='delete'
    return modus    

Get Modus like Inverter or On-Off or Two-Stages by comparing the thermal Power output at different temperature levels:

-7/34 |  2/30  |  7/27  |  12/24


assumptions for On-Off Heatpump:
if temperature difference is bigger thermal Power output is smaller

assumptions for 2 stages Heatpump:
if temperature difference is bigger thermal Power output is smaller
but it has one Jump in between 2/30 and 7/27

Inverter: the rest

t~36s

In [ ]:
data_key = pd.read_csv(r'output/data_key_normalized.csv', delimiter=',') #read Dataframe of all models
Models=data_key['Model'].values.tolist()
Models = list(dict.fromkeys(Models))
data_keymark = data_key.rename(columns={'P_el [W]': 'P_el', 'P_th [W]': 'P_th', 'T_in [°C]': 'T_in', 'T_out [°C]': 'T_out'})
data_keymark['deltaT']=data_keymark['T_out']-data_keymark['T_in']

Moduslist=[]
for model in Models:
    try:
        P_thermal=[]
        filt1=data_keymark['T_out']==34
        Tin_minus_seven = data_keymark.loc[filt1]
        filt2=Tin_minus_seven['Model']==model
        Model_minus_seven = Tin_minus_seven[filt2]
        P_th_minus_seven = Model_minus_seven['P_th'].array[0]
        P_thermal.append(P_th_minus_seven)


        filt1=data_keymark['T_out']==30
        T_in_plus_two = data_keymark.loc[filt1]
        filt2=T_in_plus_two['Model']==model
        Model_plus_two = T_in_plus_two[filt2]
        P_th_plus_two = Model_plus_two['P_th'].array[0]
        P_thermal.append(P_th_plus_two)

        filt1=data_keymark['T_out']==27
        Tin_plus_seven = data_keymark.loc[filt1]
        filt2=Tin_plus_seven['Model']==model
        Model_plus_seven = Tin_plus_seven[filt2]
        P_th_plus_seven = Model_plus_seven['P_th'].array[0]
        P_thermal.append(P_th_plus_seven)

        filt1=data_keymark['T_out']==24
        Tin_plus_twelfe = data_keymark.loc[filt1]
        filt2=Tin_plus_twelfe['Model']==model
        Model_plus_twelfe = Tin_plus_twelfe[filt2]
        P_th_plus_twelfe = Model_plus_twelfe['P_th'].array[0]
        P_thermal.append(P_th_plus_twelfe)
        P_thermal
        Modus = get_modus(P_thermal[0],P_thermal[1],P_thermal[2],P_thermal[3])
    except:
        print(model)
    Moduslist.append(Modus)
Modusdf = pd.DataFrame()
Modusdf['Model']=Models
Modusdf['Modus']=Moduslist
Modusdf
data_key = pd.read_csv(r'output/data_key_normalized.csv', delimiter=',') #read Dataframe of all models
data_key=data_key.merge(Modusdf, how='inner', on='Model')
data_key.to_csv(r'output/data_key_modus.csv', encoding='utf-8', index=False)

In [ ]:
#data_key = data_key.loc[data_key['Modus']!='delete']
#newdf=data_key.loc[data_key['Modus']=='2-stagesLW300']
#newdf.to_csv('twostages.csv', encoding='utf-8', index=False)

In [ ]:
data_key = pd.read_csv(r'output/data_key_modus.csv', delimiter=',') #read Dataframe of all models
data_key = data_key.loc[data_key['Modus']!='delete']
filt1 = (data_key['Type'] == 'Outdoor Air/Water') & (data_key['Modus']=='Inverter')
data_key.loc[filt1, 'Group'] = 1
filt1 = (data_key['Type'] == 'Exhaust Air/Water') & (data_key['Modus']=='Inverter')
data_key.loc[filt1, 'Group'] = 1
filt1 = (data_key['Type'] == 'Brine/Water') & (data_key['Modus']=='Inverter')
data_key.loc[filt1, 'Group'] = 2
filt1 = (data_key['Type'] == 'Water/Water') & (data_key['Modus']=='Inverter')
data_key.loc[filt1, 'Group'] = 3


filt1 = (data_key['Type'] == 'Outdoor Air/Water') & (data_key['Modus']=='On-Off')
data_key.loc[filt1, 'Group'] = 4
filt1 = (data_key['Type'] == 'Exhaust Air/Water') & (data_key['Modus']=='On-Off')
data_key.loc[filt1, 'Group'] = 4
filt1 = (data_key['Type'] == 'Brine/Water') & (data_key['Modus']=='On-Off')
data_key.loc[filt1, 'Group'] = 5
filt1 = (data_key['Type'] == 'Water/Water') & (data_key['Modus']=='On-Off')
data_key.loc[filt1, 'Group'] = 6

filt1 = (data_key['Type'] == 'Outdoor Air/Water') & (data_key['Modus']=='2-Stages')
data_key.loc[filt1, 'Group'] = 7
filt1 = (data_key['Type'] == 'Exhaust Air/Water') & (data_key['Modus']=='2-Stages')
data_key.loc[filt1, 'Group'] = 7
filt1 = (data_key['Type'] == 'Brine/Water') & (data_key['Modus']=='2-Stages')
data_key.loc[filt1, 'Group'] = 8
filt1 = (data_key['Type'] == 'Water/Water') & (data_key['Modus']=='2-Stages')
data_key.loc[filt1, 'Group'] = 9
data_key.to_csv(r'output/data_key.csv', encoding='utf-8', index=False)